In [ ]:
images = images_sorted["file"].tolist() # input a list of files with appropriate file paths here

start_star_x = 1117
start_star_y = 1544
end_star_x = 824
end_star_y = 859

deltax_total = end_star_x - start_star_x
deltay_total = end_star_y - start_star_y
dividing_factor = len(ZTF_images) - 1

deltax_increment = deltax_total / dividing_factor
deltay_increment = deltay_total / dividing_factor

photometry_target = pd.DataFrame(columns = ["num", "id", "xcenter", "ycenter", "aperture_sum", "total_bkg", "aperture_sum_bkgsub"])
photometry_star = pd.DataFrame(columns = ["num", "id", "xcenter", "ycenter", "aperture_sum", "total_bkg", "aperture_sum_bkgsub"])

for i in range(len(ZTF_images)):
    star_position_x = start_star_x + deltax_increment*i
    star_position_y = start_star_y + deltay_increment*i
    target_position = (348, 971)
    positions = [target_position, (star_position_x, star_position_y)]
    
    aperture = 15
    annulus_inner = 25
    annulus_outer = 45
    
    aperture = CircularAperture(positions, r = aperture)
    annulus_aperture = CircularAnnulus(positions, r_in = annulus_inner, r_out = annulus_outer)
    img_open = fits.open(Path(ZTF_images[i]))
    img = img_open[0].data
    
    phot_table = aperture_photometry(img, aperture)
    aperstats = ApertureStats(img, annulus_aperture)
    bkg_mean = aperstats.mean
    aperture_area = aperture.area_overlap(img)
    total_bkg = bkg_mean * aperture_area
    phot_table['total_bkg'] = total_bkg
    phot_bkgsub = phot_table['aperture_sum'] - total_bkg
    phot_table['aperture_sum_bkgsub'] = phot_bkgsub

    for col in phot_table.colnames:
        phot_table[col].info.format = '%.8g'  # for consistent table output
    
    phot_dataframe = pd.DataFrame(np.array(phot_table))
    photometry_target.loc[i] = phot_dataframe.iloc[0]
    photometry_star.loc[i] = phot_dataframe.iloc[1]
    ## add more here if there are more than 2 apertures
    
combined_data = pd.DataFrame(columns = ["target_subtracted_counts", "star_subtracted_counts", "num"])
combined_data["target_subtracted_counts"] = photometry_target["aperture_sum_bkgsub"]
combined_data["star_subtracted_counts"] = photometry_star["aperture_sum_bkgsub"]
combined_data["differential"] = combined_data["star_subtracted_counts"] - combined_data["target_subtracted_counts"]
combined_data["uncalibrated_star_mag"] = -2.5 * np.log10(combined_data["star_subtracted_counts"])
combined_data["uncalibrated_target_mag"] = -2.5 * np.log10(combined_data["target_subtracted_counts"])
combined_data["differential_mag"] = combined_data["uncalibrated_star_mag"] - combined_data["uncalibrated_target_mag"]
for i in range(len(combined_data)):
    combined_data["num"].loc[i] = i
plt.scatter(combined_data["num"], combined_data["differential"])